<a target="_blank" href="https://colab.research.google.com/github/amanichopra/sap-genai-hub/blob/main/orchestration_chatbot.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Preparation

## Install Libraries

In [7]:
!pip install -U "generative-ai-hub-sdk[all]"
!pip install "numpy<2.0.0" --force-reinstall

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 2.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Now, make sure to reset the runtime. In Google Colab, you can do this by clicking `Runtime` and `Restart Session`, as shown here:

<img src="assets/colab_restart_session.png" style="width:500px">

Now, you can continue by running the below cells. The packages have already been installed into the runtime before restarting.

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to read the [Generative AI Hub SDK docs](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/index.html) for more details. Below you will find an example for authenticating via environment variables using this very notebook. Ensure to store credentials in a file called `env_vars.env` file for the below command to work. If using Google Colab, you can place this file in the project folder by clicking the folder icon on the left and dropping the file in the workspace as shown:

<img src="./assets/upload_env.png" style="width:500px">

In [8]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='env_vars.env')

True

# Building a Chatbot Using the Orchestration Service

We will now combine templating and content filtering in a chatbot. Additionally we will make use of the orchestration services history capability.

We start with building the chatbot class. This class will be responsible for providing the template for the user query and managing the history. Note that the `chat` method uses the `history` parameter to send along the history to the orchestration service.

In [3]:
from typing import List

from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService


class OrchestrationChatbot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.service.config.template = Template(
            messages=[
                SystemMessage("You are a helpful chatbot assistant."),
                UserMessage("{{?user_query}}"),
            ],
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

Next up we will create an orchestration configuration. Next to the model parameter we also pass a content filter for input and output to ensure a nice conversation.

In [4]:
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold
from gen_ai_hub.orchestration.models.llama_guard_3_filter import LlamaGuard38bFilter
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.content_filtering import ContentFiltering, InputFiltering

input_content_filter_azure = AzureContentFilter(
    hate=AzureThreshold.ALLOW_SAFE,
    sexual=AzureThreshold.ALLOW_SAFE,
    self_harm=AzureThreshold.ALLOW_SAFE,
    violence=AzureThreshold.ALLOW_SAFE,
)

input_content_filter_llama = LlamaGuard38bFilter(violent_crimes=False, 
                                                 non_violent_crimes=False, 
                                                 sex_crimes=False, 
                                                 child_exploitation=False, 
                                                 defamation=False, 
                                                 specialized_advice=False, 
                                                 privacy=False, 
                                                 intellectual_property=False, 
                                                 indiscriminate_weapons=False, 
                                                 hate=True, 
                                                 self_harm=False, 
                                                 sexual_content=False, 
                                                 elections=False, 
                                                 code_interpreter_abuse=False)

config = OrchestrationConfig(
    llm=LLM('gemini-1.5-flash'),
    filtering=ContentFiltering(
        input_filtering=InputFiltering(filters=[input_content_filter_azure, input_content_filter_llama])
    ),
    template=None
)

orchestration_service = OrchestrationService(
    deployment_id=os.environ["AICORE_ORCH_DEPLOYMENT_ID"],
    config=config,
)

Lets chat with our bot and test if it is able to recall a conversation!

In [5]:
bot = OrchestrationChatbot(orchestration_service=orchestration_service)

In [16]:
print(bot.chat("Hello, how are you? My name is Aman."))

Hello Aman! I'm doing well, thank you for asking. It's nice to meet you.  How are you today?



In [17]:
print(bot.chat("What's the weather like today?"))

I don't have access to real-time information, including weather forecasts. To find out what the weather is like today, please check a local weather website or app.



In [18]:
print(bot.chat("Can you remember what I first asked you?"))

Yes, your first question was "Hello, how are you?"



In [19]:
print(bot.chat("What about my name?"))

You told me your name is Aman.



# Summary

In this exercise, you got hands-on experience on how to use orchestration in complex scenarios. Next, you can try building a Gen AI application with Orchestration in [Exercise 5](./orchestration_app/README.md) or continue with building a RAG pipeline in [Exercise 6](./RAG.ipynb).